In [1]:
import spatialdata
import skimage.io as io
import numpy as np
import opendvp
import pandas as pd
import anndata as ad
import geopandas as gpd
from shapely import Polygon
import ast
from typing import Sequence

import anndata as ad
import geopandas as gpd
import numpy as np
import pandas as pd

from opendvp.utils import logger

/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/dev-sdata/lib/python3.13/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/dev-sdata/lib/python3.13/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/dev-sdata/lib/python3.13/site-packages/spatialdata/_core/query/relational_query.py:532: FutureWarning: functools.pa

In [2]:
from typing import Literal

import anndata as ad
import numpy as np
import pandas as pd
import pytest

from opendvp.tl.filter_features_byNaNs import filter_features_byNaNs

In [3]:
def create_test_adata(
    n_obs=10,
    n_vars=5,
    nan_pattern : Literal['random', 'structured', 'all_valid', 'all_nan'] | None = None,  
    grouping_col=None,
    var_cols=None
):
    """Creates a dummy AnnData object with specified dimensions and NaN patterns."""
    X = np.random.rand(n_obs, n_vars) * 10

    if nan_pattern == 'random':
        nan_indices = np.random.choice(n_obs * n_vars, int(n_obs * n_vars * 0.2), replace=False)
        X.flat[nan_indices] = np.nan
    elif nan_pattern == 'structured':
        # Introduce structured NaNs for predictable testing
        X[:n_obs//2, 0] = np.nan # Protein 0 has NaNs in first half of samples
        X[n_obs//2:, 1] = np.nan # Protein 1 has NaNs in second half of samples
        X[:, 2] = np.nan # Protein 2 is all NaNs
        X[:, 3] = 1.0 # Protein 3 is all valid
        X[:, 4] = 1.0 # Protein 4 is all valid
    elif nan_pattern == 'all_valid':
        X = np.ones((n_obs, n_vars))
    elif nan_pattern == 'all_nan':
        X = np.full((n_obs, n_vars), np.nan)

    obs = pd.DataFrame(index=[f'cell_{i}' for i in range(n_obs)])
    if grouping_col:
        obs[grouping_col] = ['groupA'] * (n_obs // 2) + ['groupB'] * (n_obs - n_obs // 2)
    
    var_data = {'original_col_1': [f'gene_{i}' for i in range(n_vars)],
                'original_col_2': np.arange(n_vars)}
    if var_cols:
        var_data.update(var_cols)
    var = pd.DataFrame(var_data, index=[f'protein_{i}' for i in range(n_vars)])

    adata = ad.AnnData(X=X, obs=obs, var=var)
    return adata

In [9]:
adata = create_test_adata(n_obs=5, n_vars=5, nan_pattern="random")
adata.X

array([[3.70901514, 1.06302868, 4.67816553, 1.51995282,        nan],
       [9.61504541, 6.79896676, 9.16804934, 8.47162031, 0.06167912],
       [       nan, 0.88580514,        nan, 4.54793265, 3.40813459],
       [3.47695189,        nan, 7.57355187, 7.62349755, 5.94144452],
       [7.58631469, 5.05908974, 1.60172298, 8.08754087,        nan]])

In [14]:
adata = create_test_adata(n_obs=10, n_vars=5, nan_pattern="structured")
adata.X

array([[       nan, 2.17249272,        nan, 1.        , 1.        ],
       [       nan, 1.80508105,        nan, 1.        , 1.        ],
       [       nan, 4.12400906,        nan, 1.        , 1.        ],
       [       nan, 1.19916888,        nan, 1.        , 1.        ],
       [       nan, 7.72565344,        nan, 1.        , 1.        ],
       [8.78656139,        nan,        nan, 1.        , 1.        ],
       [3.10320042,        nan,        nan, 1.        , 1.        ],
       [3.75802253,        nan,        nan, 1.        , 1.        ],
       [9.83712897,        nan,        nan, 1.        , 1.        ],
       [7.30276396,        nan,        nan, 1.        , 1.        ]])

array([[3.75427689,        nan, 7.05459632, 9.79896367,        nan],
       [2.19775071, 8.30591211,        nan, 7.3626556 , 1.97758067],
       [       nan, 0.46552613, 7.12953151, 7.96498521, 9.66215719],
       [7.57196055, 9.47889411, 1.54342085, 1.17498436, 8.41366918],
       [0.70874143,        nan, 9.5174399 , 4.89150773, 9.97833865]])

In [16]:
x= 5

In [17]:
np.isnan(x)

np.False_

In [18]:
adata

AnnData object with n_obs × n_vars = 10 × 5
    var: 'original_col_1', 'original_col_2'

In [19]:
qc_metrics = pd.DataFrame(index=adata.var_names)
qc_metrics['imputed_values'] = pd.Series(index=qc_metrics.index, dtype='object')

In [20]:
qc_metrics

,imputed_values
protein_0,NaN
protein_1,NaN
protein_2,NaN
protein_3,NaN
protein_4,NaN


In [21]:
qc_metrics.dtypes

imputed_values    object
dtype: object

In [ ]:
x = np.array([1,2,3])

In [23]:
qc_metrics.loc["protein_0","imputed_values"] = x

In [24]:
qc_metrics

,imputed_values
protein_0,"[1, 2, 3]"
protein_1,NaN
protein_2,NaN
protein_3,NaN
protein_4,NaN


In [ ]:
import anndata as ad
import numpy as np
import pandas as pd

from opendvp.utils import logger


def impute_gaussian(
    adata: ad.AnnData,
    mean_shift: float = -1.8,
    std_dev_shift: float = 0.3,
    perSample: bool = False,
    layer_key: str = 'unimputed',
    uns_key: str = 'impute_gaussian_qc_metrics'
) -> ad.AnnData:
    """Impute missing values in an AnnData object using a Gaussian distribution.

    This function imputes missing values in the data matrix using a Gaussian distribution, with the mean shifted and
    the standard deviation scaled. Imputation can be performed per protein (column) or per sample (row).

    The original, un-imputed data matrix is stored in `adata.layers`.
    A DataFrame with quality control metrics for the imputation is stored in `adata.uns`. The QC metrics include
    the number of imputed values, the mean and standard deviation used for imputation, and a numpy array of the
    imputed values themselves for each feature.

    Parameters
    ----------
    adata : ad.AnnData
        AnnData object with missing values to impute.
    mean_shift : float, default -1.8
        Number of standard deviations to shift the mean of the Gaussian distribution.
    std_dev_shift : float, default 0.3
        Factor to scale the standard deviation of the Gaussian distribution.
    perSample : bool, default False
        If True, impute per sample (row); if False, impute per protein (column).
    layer_key : str, default 'unimputed'
        Key under which to store the original, un-imputed data matrix in `adata.layers`.
    uns_key : str, default 'impute_gaussian_qc_metrics'
        Key under which to store the imputation QC metrics DataFrame in `adata.uns`.

    Returns:
    -------
    ad.AnnData
        AnnData object with imputed values in `.X`, the original matrix in `.layers[layer_key]`,
        and QC metrics in `.uns[uns_key]`.
    """
    adata_copy = adata.copy()

    # Store the original data in a new layer
    logger.info(f"Storing original data in `adata.layers['{layer_key}']`.")
    adata_copy.layers[layer_key] = adata_copy.X.copy()

    # Ensure dense array for DataFrame construction
    data = np.asarray(adata_copy.X)
    impute_df = pd.DataFrame(data=data, columns=adata_copy.var.index, index=adata_copy.obs_names)

    if perSample:
        logger.info("Imputation with Gaussian distribution PER SAMPLE")
        impute_df = impute_df.T
    else:
        logger.info("Imputation with Gaussian distribution PER PROTEIN")

    # Initialize QC metrics DataFrame
    qc_metrics = pd.DataFrame(
        index=impute_df.columns,
        columns=['n_imputed', 'imputation_mean', 'imputation_stddev'],
    )
    # Add a column to store the imputed values, initialized to hold empty arrays
    qc_metrics['imputed_values'] = pd.Series(index=qc_metrics.index, dtype='object')
    # qc_metrics['imputed_values'] = [np.array([]) for _ in range(len(qc_metrics))]

    #seems to not work
    # qc_metrics['imputed_values'] = pd.Series(
    #     [np.array([]) for _ in range(len(qc_metrics))], 
    #     index=qc_metrics.index, dtype='object')


    logger.info(f'Mean number of missing values per sample: '
                f'{round(impute_df.isna().sum(axis=1).mean(),2)} out of {impute_df.shape[1]} proteins')
    logger.info(f'Mean number of missing values per protein: '
                f'{round(impute_df.isna().sum(axis=0).mean(),2)} out of {impute_df.shape[0]} samples')

    for col in impute_df.columns:
        col_mean = np.nanmean(impute_df[col])
        col_stddev = np.nanstd(impute_df[col], ddof=1)
        nan_mask = impute_df[col].isna()
        num_nans = nan_mask.sum()

        # Store QC metrics
        qc_metrics.loc[col, 'n_imputed'] = num_nans
        qc_metrics.loc[col, 'imputation_mean'] = col_mean
        qc_metrics.loc[col, 'imputation_stddev'] = col_stddev

        if num_nans > 0:
            shifted_random_values = np.random.normal(
                loc=(col_mean + (mean_shift * col_stddev)),
                scale=(col_stddev * std_dev_shift),size=num_nans)
            
            qc_metrics.loc[col, 'imputed_values'] = shifted_random_values
            
            impute_df.loc[nan_mask, col] = shifted_random_values
            logger.debug(f"Imputed {num_nans} NaNs in column '{col}' with mean={col_mean:.2f}, std={col_stddev:.2f}")

    if perSample:
        impute_df = impute_df.T

    if (impute_df < 0).any().any():
        logger.warning("Negative values found after imputation. Impute log-transformed data instead.")

    adata_copy.X = impute_df.to_numpy()
    adata_copy.uns[uns_key] = qc_metrics
    logger.info(f"Imputation complete. QC metrics stored in `adata.uns['{uns_key}']`.")

    return adata_copy

In [49]:
# Helper function to create a dummy AnnData object for testing
def create_test_adata(
    n_obs=10,
    n_vars=5,
    nan_pattern : Literal['random', 'structured', 'all_valid', 'all_nan'] | None = None,  
    grouping_col=None,
    var_cols=None
):
    """Creates a dummy AnnData object with specified dimensions and NaN patterns."""
    X = np.random.rand(n_obs, n_vars) * 10

    if nan_pattern == 'random':
        nan_indices = np.random.choice(n_obs * n_vars, int(n_obs * n_vars * 0.2), replace=False)
        X.flat[nan_indices] = np.nan
    elif nan_pattern == 'structured':
        # Introduce structured NaNs for predictable testing
        X[:n_obs//2, 0] = np.nan # Protein 0 has NaNs in first half of samples
        X[n_obs//2:, 1] = np.nan # Protein 1 has NaNs in second half of samples
        X[:, 2] = np.nan # Protein 2 is all NaNs
        X[:, 3] = 1.0 # Protein 3 is all valid
        X[:, 4] = 1.0 # Protein 4 is all valid
    elif nan_pattern == 'all_valid':
        X = np.ones((n_obs, n_vars))
    elif nan_pattern == 'all_nan':
        X = np.full((n_obs, n_vars), np.nan)

    obs = pd.DataFrame(index=[f'cell_{i}' for i in range(n_obs)])
    if grouping_col:
        obs[grouping_col] = ['groupA'] * (n_obs // 2) + ['groupB'] * (n_obs - n_obs // 2)
    
    var_data = {'original_col_1': [f'gene_{i}' for i in range(n_vars)],
                'original_col_2': np.arange(n_vars)}
    if var_cols:
        var_data.update(var_cols)
    var = pd.DataFrame(var_data, index=[f'protein_{i}' for i in range(n_vars)])

    adata = ad.AnnData(X=X, obs=obs, var=var)
    return adata

In [50]:
def test_impute_per_protein_default():
    """Test default imputation (per protein) and QC storage."""
    adata = create_test_adata(n_obs=10, n_vars=5, nan_pattern='structured')
    original_X = adata.X.copy()

    # protein_0 has 5 NaNs, protein_1 has 5 NaNs, protein_2 has 10 NaNs, p3 and p4 no NaNs

    imputed_adata = impute_gaussian(adata)
    
    # 1. Check that no NaNs remain in .X (except for all-NaN columns)
    # protein_2 was all NaNs, so it will be imputed with NaNs.
    assert not np.isnan(imputed_adata.X[:, :2]).any()
    assert np.isnan(imputed_adata.X[:, 2]).all()

    # 2. Check that the original data is stored in layers
    assert 'unimputed' in imputed_adata.layers
    assert np.allclose(imputed_adata.layers['unimputed'], original_X, equal_nan=True)

    # 3. Check that QC metrics are stored in uns
    assert 'impute_gaussian_qc_metrics' in imputed_adata.uns
    qc_df = imputed_adata.uns['impute_gaussian_qc_metrics']
    assert isinstance(qc_df, pd.DataFrame)
    assert qc_df.index.equals(adata.var_names)
    expected_cols = ['n_imputed', 'imputation_mean', 'imputation_stddev', 'imputed_values']
    assert all(col in qc_df.columns for col in expected_cols)

    # 4. Verify QC content for a specific protein (protein_0)
    assert qc_df.loc['protein_0', 'n_imputed'] == 5
    assert len(qc_df.loc['protein_0', 'imputed_values']) == 5
    assert isinstance(qc_df.loc['protein_0', 'imputed_values'], np.ndarray)

    # 5. Verify QC for all-NaN protein (protein_2)
    assert qc_df.loc['protein_2', 'n_imputed'] == 10
    assert np.isnan(qc_df.loc['protein_2', 'imputation_mean'])
    assert np.isnan(qc_df.loc['protein_2', 'imputation_stddev'])
    assert np.isnan(qc_df.loc['protein_2', 'imputed_values']).all()

In [51]:
adata = create_test_adata(n_obs=10, n_vars=5, nan_pattern='structured')

In [52]:
adata.X

array([[       nan, 7.75379581,        nan, 1.        , 1.        ],
       [       nan, 9.03488776,        nan, 1.        , 1.        ],
       [       nan, 8.42742683,        nan, 1.        , 1.        ],
       [       nan, 1.59728843,        nan, 1.        , 1.        ],
       [       nan, 3.92711211,        nan, 1.        , 1.        ],
       [3.16188169,        nan,        nan, 1.        , 1.        ],
       [0.48347712,        nan,        nan, 1.        , 1.        ],
       [8.94342665,        nan,        nan, 1.        , 1.        ],
       [9.71564164,        nan,        nan, 1.        , 1.        ],
       [8.44210062,        nan,        nan, 1.        , 1.        ]])

In [53]:
imputed_adata = impute_gaussian(adata)

18:19:01.29 | INFO | Storing original data in `adata.layers['unimputed']`.
18:19:01.29 | INFO | Imputation with Gaussian distribution PER PROTEIN
18:19:01.29 | INFO | Mean number of missing values per sample: 2.0 out of 5 proteins
18:19:01.29 | INFO | Mean number of missing values per protein: 4.0 out of 10 samples


ValueError: Must have equal len keys and value when setting with an iterable

In [40]:
imputed_adata.X

array([[8.49705618, 7.3581895 , 4.20559146, 9.91206742, 6.58345134],
       [0.76113295, 5.12764459, 3.0216116 , 4.11967499, 5.68527244],
       [0.05210005, 1.04339021, 2.68151478, 6.96881004, 2.10890592],
       [7.17543248, 2.15492011, 4.46189994, 1.73034951, 4.41563864],
       [3.82016288, 0.29764584, 0.4347652 , 3.46780576, 6.99858562],
       [3.05175812, 2.69534419, 2.29500328, 7.63155378, 1.81213786],
       [6.08325215, 5.99032625, 4.2419341 , 8.66353157, 9.2658506 ],
       [5.45969867, 1.89974184, 0.27863853, 7.09041511, 4.46405983],
       [9.90320926, 5.31066379, 9.72723725, 3.02770136, 7.68156629],
       [2.85363228, 4.09439381, 2.03896314, 4.72104249, 8.00886561]])

In [36]:
test_impute_per_protein_default()

18:16:47.25 | INFO | Storing original data in `adata.layers['unimputed']`.
18:16:47.25 | INFO | Imputation with Gaussian distribution PER PROTEIN
18:16:47.25 | INFO | Mean number of missing values per sample: 0.0 out of 5 proteins
18:16:47.25 | INFO | Mean number of missing values per protein: 0.0 out of 10 samples
18:16:47.25 | INFO | Imputation complete. QC metrics stored in `adata.uns['impute_gaussian_qc_metrics']`.


AssertionError: 

In [54]:
adata_copy = adata.copy()

data = np.asarray(adata_copy.X)
impute_df = pd.DataFrame(data=data, columns=adata_copy.var.index, index=adata_copy.obs_names)

qc_metrics = pd.DataFrame(
        index=impute_df.columns,
        columns=['n_imputed', 'imputation_mean', 'imputation_stddev'],
    )

In [55]:
qc_metrics

,n_imputed,imputation_mean,imputation_stddev
protein_0,NaN,NaN,NaN
protein_1,NaN,NaN,NaN
protein_2,NaN,NaN,NaN
protein_3,NaN,NaN,NaN
protein_4,NaN,NaN,NaN


In [56]:
qc_metrics['imputed_values'] = pd.Series(index=qc_metrics.index, dtype='object')

In [57]:
qc_metrics

,n_imputed,imputation_mean,imputation_stddev,imputed_values
protein_0,NaN,NaN,NaN,NaN
protein_1,NaN,NaN,NaN,NaN
protein_2,NaN,NaN,NaN,NaN
protein_3,NaN,NaN,NaN,NaN
protein_4,NaN,NaN,NaN,NaN


In [58]:
qc_metrics.dtypes

n_imputed            object
imputation_mean      object
imputation_stddev    object
imputed_values       object
dtype: object

In [65]:
df = pd.DataFrame(columns=[1])
df[1] = df[1].astype(object)
df.loc[1, 1] = np.array([5, 6, 7, 8])
df

,1
1,"[5, 6, 7, 8]"


In [69]:
mean_shift = -1.8
std_dev_shift = 0.3

for col in impute_df.columns:
        
        print(col)
        print(qc_metrics.index)
        assert col in qc_metrics.index

        col_mean = np.nanmean(impute_df[col])
        col_stddev = np.nanstd(impute_df[col], ddof=1)
        nan_mask = impute_df[col].isna()
        num_nans = nan_mask.sum()

        # Store QC metrics
        qc_metrics.loc[col, 'n_imputed'] = num_nans
        qc_metrics.loc[col, 'imputation_mean'] = col_mean
        qc_metrics.loc[col, 'imputation_stddev'] = col_stddev

        if num_nans > 0:
            shifted_random_values = np.random.normal(
                loc=(col_mean + (mean_shift * col_stddev)),
                scale=(col_stddev * std_dev_shift),size=num_nans)
            
            print(len(shifted_random_values))
            print(shifted_random_values)
            print(type(shifted_random_values))

            qc_metrics.at[col, 'imputed_values'] = shifted_random_values
            
            impute_df.loc[nan_mask, col] = shifted_random_values
            logger.debug(f"Imputed {num_nans} NaNs in column '{col}' with mean={col_mean:.2f}, std={col_stddev:.2f}")

protein_0
Index(['protein_0', 'protein_1', 'protein_2', 'protein_3', 'protein_4'], dtype='object')
5
[ 0.42944793 -0.69866389 -0.33605261 -1.65708508 -2.13870934]
<class 'numpy.ndarray'>
protein_1
Index(['protein_0', 'protein_1', 'protein_2', 'protein_3', 'protein_4'], dtype='object')
5
[ 0.15489287 -0.25266274 -0.09080634  0.47524837  0.83365573]
<class 'numpy.ndarray'>
protein_2
Index(['protein_0', 'protein_1', 'protein_2', 'protein_3', 'protein_4'], dtype='object')
10
[nan nan nan nan nan nan nan nan nan nan]
<class 'numpy.ndarray'>
protein_3
Index(['protein_0', 'protein_1', 'protein_2', 'protein_3', 'protein_4'], dtype='object')
protein_4
Index(['protein_0', 'protein_1', 'protein_2', 'protein_3', 'protein_4'], dtype='object')


/var/folders/x7/grkjlk8s223dy6234rnz1885mxz2_6/T/ipykernel_36553/503741152.py:10: RuntimeWarning: Mean of empty slice
  col_mean = np.nanmean(impute_df[col])
/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/dev-sdata/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [71]:
import anndata as ad
import numpy as np
import pandas as pd

from opendvp.utils import logger


def impute_gaussian(
    adata: ad.AnnData,
    mean_shift: float = -1.8,
    std_dev_shift: float = 0.3,
    perSample: bool = False,
    layer_key: str = 'unimputed',
    uns_key: str = 'impute_gaussian_qc_metrics'
) -> ad.AnnData:
    """Impute missing values in an AnnData object using a Gaussian distribution.

    This function imputes missing values in the data matrix using a Gaussian distribution, with the mean shifted and
    the standard deviation scaled. Imputation can be performed per protein (column) or per sample (row).

    The original, un-imputed data matrix is stored in `adata.layers`.
    A DataFrame with quality control metrics for the imputation is stored in `adata.uns`. The QC metrics include
    the number of imputed values, the mean and standard deviation used for imputation, and a numpy array of the
    imputed values themselves for each feature.

    Parameters
    ----------
    adata : ad.AnnData
        AnnData object with missing values to impute.
    mean_shift : float, default -1.8
        Number of standard deviations to shift the mean of the Gaussian distribution.
    std_dev_shift : float, default 0.3
        Factor to scale the standard deviation of the Gaussian distribution.
    perSample : bool, default False
        If True, impute per sample (row); if False, impute per protein (column).
    layer_key : str, default 'unimputed'
        Key under which to store the original, un-imputed data matrix in `adata.layers`.
    uns_key : str, default 'impute_gaussian_qc_metrics'
        Key under which to store the imputation QC metrics DataFrame in `adata.uns`.

    Returns:
    -------
    ad.AnnData
        AnnData object with imputed values in `.X`, the original matrix in `.layers[layer_key]`,
        and QC metrics in `.uns[uns_key]`.
    """
    adata_copy = adata.copy()

    # Store the original data in a new layer
    logger.info(f"Storing original data in `adata.layers['{layer_key}']`.")
    adata_copy.layers[layer_key] = adata_copy.X.copy()

    # Ensure dense array for DataFrame construction
    data = np.asarray(adata_copy.X)
    impute_df = pd.DataFrame(data=data, columns=adata_copy.var.index, index=adata_copy.obs_names)

    if perSample:
        logger.info("Imputation with Gaussian distribution PER SAMPLE")
        impute_df = impute_df.T
    else:
        logger.info("Imputation with Gaussian distribution PER PROTEIN")

    # Initialize QC metrics DataFrame
    qc_metrics = pd.DataFrame(
        index=impute_df.columns, columns=['n_imputed', 'imputation_mean', 'imputation_stddev'])
    qc_metrics['imputed_values'] = pd.Series(index=qc_metrics.index, dtype='object')

    logger.info(f'Mean number of missing values per sample: '
                f'{round(impute_df.isna().sum(axis=1).mean(),2)} out of {impute_df.shape[1]} proteins')
    logger.info(f'Mean number of missing values per protein: '
                f'{round(impute_df.isna().sum(axis=0).mean(),2)} out of {impute_df.shape[0]} samples')

    for col in impute_df.columns:
        col_mean = np.nanmean(impute_df[col])
        col_stddev = np.nanstd(impute_df[col], ddof=1)
        nan_mask = impute_df[col].isna()
        num_nans = nan_mask.sum()

        # Store QC metrics
        qc_metrics.loc[col, 'n_imputed'] = num_nans
        qc_metrics.loc[col, 'imputation_mean'] = col_mean
        qc_metrics.loc[col, 'imputation_stddev'] = col_stddev

        if num_nans > 0:
            shifted_random_values = np.random.normal(
                loc=(col_mean + (mean_shift * col_stddev)),
                scale=(col_stddev * std_dev_shift),size=num_nans)
            
            qc_metrics.at[col, 'imputed_values'] = shifted_random_values  # noqa: PD008
            impute_df.loc[nan_mask, col] = shifted_random_values
            logger.debug(f"Imputed {num_nans} NaNs in column '{col}' with mean={col_mean:.2f}, std={col_stddev:.2f}")

    if perSample:
        impute_df = impute_df.T

    if (impute_df < 0).any().any():
        logger.warning("Negative values found after imputation. Impute log-transformed data instead.")

    adata_copy.X = impute_df.to_numpy()
    adata_copy.uns[uns_key] = qc_metrics
    logger.info(f"Imputation complete. QC metrics stored in `adata.uns['{uns_key}']`.")

    return adata_copy

In [76]:
def test_no_nans_no_change():
    """Test that data with no NaNs is not changed."""
    adata = create_test_adata(n_obs=10, n_vars=5, nan_pattern='all_valid')
    original_X = adata.X.copy()

    imputed_adata = impute_gaussian(adata)

    # .X should be unchanged
    assert np.array_equal(imputed_adata.X, original_X)

    # QC metrics should reflect no imputation
    qc_df = imputed_adata.uns['impute_gaussian_qc_metrics']
    return qc_df
    # assert qc_df['n_imputed'].sum() == 0
    # assert all(len(arr) == 0 for arr in qc_df['imputed_values'])

In [77]:
df = test_no_nans_no_change()

18:31:38.15 | INFO | Storing original data in `adata.layers['unimputed']`.
18:31:38.15 | INFO | Imputation with Gaussian distribution PER PROTEIN
18:31:38.15 | INFO | Mean number of missing values per sample: 0.0 out of 5 proteins
18:31:38.15 | INFO | Mean number of missing values per protein: 0.0 out of 10 samples
18:31:38.16 | INFO | Imputation complete. QC metrics stored in `adata.uns['impute_gaussian_qc_metrics']`.


In [78]:
df

,n_imputed,imputation_mean,imputation_stddev,imputed_values
protein_0,0,1.0,0.0,NaN
protein_1,0,1.0,0.0,NaN
protein_2,0,1.0,0.0,NaN
protein_3,0,1.0,0.0,NaN
protein_4,0,1.0,0.0,NaN
